In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("train.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
df.drop(['PassengerId','Name','Ticket','Cabin','Embarked','SibSp','Parch'],axis=1,inplace=True)

In [4]:
df.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [5]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
Fare          0
dtype: int64

In [6]:
df['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [7]:
df['Age'].fillna(df['Age'].mean(),inplace=True)

In [8]:
df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
Fare        0
dtype: int64

In [9]:
l_sex_dummies=pd.get_dummies(df['Sex'],drop_first=True)

In [10]:
df = pd.concat([df,l_sex_dummies],axis=1)

In [11]:
df.head()

,Survived,Pclass,Sex,Age,Fare,male
0,0,3,male,22.0,7.2500,1
1,1,1,female,38.0,71.2833,0
2,1,3,female,26.0,7.9250,0
3,1,1,female,35.0,53.1000,0
4,0,3,male,35.0,8.0500,1


In [12]:
from sklearn.preprocessing import StandardScaler
sts = StandardScaler()

In [13]:
feature_scale = ['Age','Fare']
df[feature_scale]=sts.fit_transform(df[feature_scale])

In [14]:
df.head()

,Survived,Pclass,Sex,Age,Fare,male
0,0,3,male,-0.592481,-0.502445,1
1,1,1,female,0.638789,0.786845,0
2,1,3,female,-0.284663,-0.488854,0
3,1,1,female,0.407926,0.420730,0
4,0,3,male,0.407926,-0.486337,1


In [15]:
df=df.drop(['Sex'],axis=1)

In [16]:
df.dtypes

Survived      int64
Pclass        int64
Age         float64
Fare        float64
male          uint8
dtype: object

In [17]:
df

,Survived,Pclass,Age,Fare,male
0,0,3,-0.592481,-0.502445,1
1,1,1,0.638789,0.786845,0
2,1,3,-0.284663,-0.488854,0
3,1,1,0.407926,0.420730,0
4,0,3,0.407926,-0.486337,1
...,...,...,...,...,...
886,0,2,-0.207709,-0.386671,1
887,1,1,-0.823344,-0.044381,0
888,0,3,0.000000,-0.176263,0
889,1,1,-0.284663,-0.044381,1


In [18]:
x=df.drop(['Survived'],axis=1)
y=df['Survived']

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [20]:
#create param
model_param = {
    'DecisionTreeClassifier':{
        'model':DecisionTreeClassifier(),
        'param':{
            'criterion':['gini','entropy']
        }
    }, 
        'KNeighborsClassifier':{
        'model':KNeighborsClassifier(),
        'param':{
            'n_neighbors':[5,10,15,20,25]
        }
    },
        'SVC':{
        'model':SVC(),
        'param':{
            'kernel':['rbf','linear','sigmoid'],
            'C':[0.1,1,10,100]
        }
    }
       
}

In [21]:
scores=[]
for model_name,mp in model_param.items():
    model_selection=GridSearchCV(estimator=mp['model'],param_grid=mp['param'],cv=5,return_train_score=False)
    model_selection.fit(x,y)
    scores.append({
        'model':model_name,
        'best_score':model_selection.best_score_,
        'best_params':model_selection.best_params_
    })

In [22]:
df_model_score=pd.DataFrame(scores,columns=['model','best_score','best_params'])
df_model_score

,model,best_score,best_params
0,DecisionTreeClassifier,0.771113,{'criterion': 'gini'}
1,KNeighborsClassifier,0.802492,{'n_neighbors': 5}
2,SVC,0.811481,"{'C': 100, 'kernel': 'rbf'}"


In [23]:
model_svc=SVC(C=100,kernel='rbf')

In [24]:
model_svc.fit(x,y)

SVC(C=100)

In [25]:
df2=pd.read_csv('test.csv')

In [26]:
df2.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [33]:
df2.dtypes


Pclass      int64
Sex        object
Age       float64
Fare      float64
dtype: object

In [27]:
df3=df2.drop(['PassengerId','Name','Ticket','Cabin','Embarked','SibSp','Parch'],axis=1,inplace=True)

In [34]:
df2.isnull().sum()

Pclass     0
Sex        0
Age       86
Fare       1
dtype: int64

In [35]:
df2.isnull().sum()

Pclass     0
Sex        0
Age       86
Fare       1
dtype: int64

In [36]:
df2['Age'].fillna(df['Age'].mean(),inplace=True)
df2['Fare'].fillna(df['Fare'].mean(),inplace=True)

In [37]:
l_sex_dummies=pd.get_dummies(df2['Sex'],drop_first=True)
df2=pd.concat([df2,l_sex_dummies],axis=1)
df2.drop(['Sex'],axis=1,inplace=True)

In [38]:
df2.head()

,Pclass,Age,Fare,male
0,3,34.5,7.8292,1
1,3,47.0,7.0000,0
2,2,62.0,9.6875,1
3,3,27.0,8.6625,1
4,3,22.0,12.2875,0


In [39]:
df2[feature_scale]=sts.fit_transform(df2[feature_scale])

In [40]:
df2.head()

,Pclass,Age,Fare,male
0,3,0.594801,-0.496637,1
1,3,1.305896,-0.511497,0
2,2,2.159209,-0.463335,1
3,3,0.168145,-0.481704,1
4,3,-0.116293,-0.416740,0


In [43]:
y_predicted=model_svc.predict(df2)

In [47]:
df3=pd.read_csv('test.csv')
df3

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [48]:
submission=pd.DataFrame({
    'PassengerId':df3['PassengerId'],
    'Survived':y_predicted
})

In [49]:
submission.to_csv('titanic_submission.csv',index=False)

In [50]:
df4=pd.read_csv("titanic_submission.csv")
df4

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
